<a href="https://colab.research.google.com/github/Nazhan99/Tensorflow_notes/blob/main/05_Transfer_learning_in_tensorflow_part_2_fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Transfer learning with TensorFlow part 2: Fine-Tuning

In [1]:
!nvidia-smi

Mon Feb 21 03:04:57 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P8    30W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

#Creating helper function

In [2]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2022-02-21 03:04:58--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2022-02-21 03:04:58 (83.0 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [3]:
#import helper function we are going to use in this notebook
from helper_functions import create_tensorboard_callback, plot_loss_curves, unzip_data,  walk_through_dir

>if you are running this notebook in Google colab, when it times out colab will delete "helper_function".

## Get some data

use pretrained model using tf.keras.application and apply them to our own probelm (recognizing images of food).

In [4]:
# get 10% of training data of 10 fooc classes
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

unzip_data("10_food_classes_10_percent.zip")

--2022-02-21 03:05:01--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.120.128, 74.125.126.128, 74.125.132.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.120.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip’

10_food_classes_10_ 100%[===================>] 160.74M   105MB/s    in 1.5s    

2022-02-21 03:05:03 (105 MB/s) - ‘10_food_classes_10_percent.zip’ saved [168546183/168546183]



In [5]:
#check out how many images and subdirectories are in our dataset
walk_through_dir("10_food_classes_10_percent")

There are 2 directories and 0 images in '10_food_classes_10_percent'.
There are 10 directories and 0 images in '10_food_classes_10_percent/train'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/pizza'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/fried_rice'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/grilled_salmon'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/hamburger'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/steak'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/sushi'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/chicken_wings'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/ramen'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/ice_cream'.
There are 0 directories and 75 images in '10_food_classes_10_percent/train/c

In [6]:
#create training and test directories paths
train_dir="10_food_classes_10_percent/train"
test_dir="10_food_classes_10_percent/test"


In [7]:
import tensorflow as tf

IMG_SIZE=(224,224)
BATCH_SIZE=32

train_data_10_percent= tf.keras.preprocessing.image_dataset_from_directory(directory=train_dir,
                                                                           image_size=IMG_SIZE,
                                                                           label_mode="categorical",
                                                                           batch_size=BATCH_SIZE)

test_data_10_percent= tf.keras.preprocessing.image_dataset_from_directory(directory=test_dir,
                                                                          image_size=IMG_SIZE,
                                                                          label_mode="categorical",
                                                                          batch_size=BATCH_SIZE)

Found 750 files belonging to 10 classes.
Found 2500 files belonging to 10 classes.


In [8]:
train_data_10_percent

<BatchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 10), dtype=tf.float32, name=None))>

In [9]:
#check out the class names of our dataset
train_data_10_percent.class_names

['chicken_curry',
 'chicken_wings',
 'fried_rice',
 'grilled_salmon',
 'hamburger',
 'ice_cream',
 'pizza',
 'ramen',
 'steak',
 'sushi']

In [10]:
#see an example of a batch of data
for image, labels in train_data_10_percent.take(1):
  print(image,labels)

#need to normalize these tensors later

tf.Tensor(
[[[[2.93418369e+01 5.56989784e+01 8.16683655e+01]
   [2.06428547e+01 4.70000000e+01 7.15204086e+01]
   [8.53571415e+00 3.33265305e+01 5.58979607e+01]
   ...
   [3.98520050e+01 8.42805328e+01 1.09494797e+02]
   [2.79540596e+01 6.99540634e+01 9.49540634e+01]
   [3.42195702e+01 7.42195663e+01 1.00219566e+02]]

  [[3.05969391e+01 6.39540825e+01 9.60000000e+01]
   [3.19132633e+01 6.32040787e+01 9.24285660e+01]
   [2.15867329e+01 5.11734695e+01 7.76989822e+01]
   ...
   [4.61989594e+01 9.06274872e+01 1.17413223e+02]
   [5.01377678e+01 9.21377640e+01 1.17137764e+02]
   [4.88570366e+01 8.88570404e+01 1.14857040e+02]]

  [[3.11938782e+01 7.31224518e+01 1.13913269e+02]
   [2.61530609e+01 6.66683655e+01 1.03525513e+02]
   [2.75051003e+01 6.40050964e+01 9.81683655e+01]
   ...
   [4.66173630e+01 9.10458908e+01 1.17831627e+02]
   [5.02142868e+01 9.14285736e+01 1.18785713e+02]
   [5.03418159e+01 8.95560989e+01 1.17913246e+02]]

  ...

  [[1.34857147e+02 1.45642853e+02 1.68285736e+02]
   [1

## model 0: Building a transfer learning feature extraction model using the Keras Functional API

The sequential API is straight forward, it runs our layers in sequential order.

But the functional API gives us flexibility with our models

---



In [14]:
#1. create base model with tf.keras.applications
base_model= tf.keras.applications.EfficientNetB0(include_top=False)

#2. freeze the base model (so the underlying pre trained patterns are not updating)
base_model.trainable=False

#3. Create inputs into our model
inputs = tf.keras.layers.Input(shape=(224,224,3), name="input_layer")

#4. If uisng Resnet50V2 you will need to normalize inputs (you dont have to for EfficientNet)
#x=tf.keras.layers.experimental.preprocessing.Rescaling(1./255)(inputs)

#5. Past the input to the base_model
x=base_model(inputs)
print(f"Shape after passing inputs through base model: {x.shape}")

#6. Average pool the outputs of the base model (aggregate all the most important information, reduce number of computations)
x=tf.keras.layers.GlobalAveragePooling2D(name="global_average_pooling_layer")(x)
print(f"Shape after GlobalAveragePooling2D: {x.shape}")

#7. Create the output activation layer
outputs = tf.keras.layers.Dense(10, activation="softmax", name="output_layer")(x)

#8. combine the inputs with the outputs into a model
model_0 = tf.keras.Model(inputs, outputs)

#9. Compile for the model
model_0.compile(loss="categorical_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

#10. fit the model
history_10_percent = model_0.fit(train_data_10_percent,
                                 epochs=5,
                                 steps_per_epoch=len(train_data_10_percent),
                                 validation_data=test_data_10_percent,
                                 validation_steps=int(0.25*len(test_data_10_percent)),
                                 callbacks=[create_tensorboard_callback(dir_name="transfer_learning",
                                                                        experiment_name="10_percent_feature_extraction")])

Shape after passing inputs through base model: (None, 7, 7, 1280)
Shape after GlobalAveragePooling2D: (None, 1280)
Saving TensorBoard log files to: transfer_learning/10_percent_feature_extraction/20220221-031018
Epoch 1/5
24/24 [==============================] - 28s 457ms/step - loss: 1.8882 - accuracy: 0.4107 - val_loss: 1.3658 - val_accuracy: 0.6974
Epoch 2/5
24/24 [==============================] - 9s 358ms/step - loss: 1.1411 - accuracy: 0.7480 - val_loss: 0.9555 - val_accuracy: 0.7911
Epoch 3/5
24/24 [==============================] - 9s 362ms/step - loss: 0.8414 - accuracy: 0.8120 - val_loss: 0.7768 - val_accuracy: 0.8158
Epoch 4/5
24/24 [==============================] - 9s 359ms/step - loss: 0.6958 - accuracy: 0.8360 - val_loss: 0.6841 - val_accuracy: 0.8372
Epoch 5/5
24/24 [==============================] - 7s 269ms/step - loss: 0.5925 - accuracy: 0.8640 - val_loss: 0.6072 - val_accuracy: 0.8438


In [19]:
#evaluate on the full test dataset
model_0.evaluate(test_data_10_percent)

79/79 [==============================] - 12s 142ms/step - loss: 0.6106 - accuracy: 0.8436


[0.6105808019638062, 0.8435999751091003]

In [23]:
#check the layers in our base model
for layer_number, layer in enumerate(base_model.layers):
  print(layer_number,layer.name)

0 input_4
1 rescaling_3
2 normalization_3
3 stem_conv_pad
4 stem_conv
5 stem_bn
6 stem_activation
7 block1a_dwconv
8 block1a_bn
9 block1a_activation
10 block1a_se_squeeze
11 block1a_se_reshape
12 block1a_se_reduce
13 block1a_se_expand
14 block1a_se_excite
15 block1a_project_conv
16 block1a_project_bn
17 block2a_expand_conv
18 block2a_expand_bn
19 block2a_expand_activation
20 block2a_dwconv_pad
21 block2a_dwconv
22 block2a_bn
23 block2a_activation
24 block2a_se_squeeze
25 block2a_se_reshape
26 block2a_se_reduce
27 block2a_se_expand
28 block2a_se_excite
29 block2a_project_conv
30 block2a_project_bn
31 block2b_expand_conv
32 block2b_expand_bn
33 block2b_expand_activation
34 block2b_dwconv
35 block2b_bn
36 block2b_activation
37 block2b_se_squeeze
38 block2b_se_reshape
39 block2b_se_reduce
40 block2b_se_expand
41 block2b_se_excite
42 block2b_project_conv
43 block2b_project_bn
44 block2b_drop
45 block2b_add
46 block3a_expand_conv
47 block3a_expand_bn
48 block3a_expand_activation
49 block3a_d

In [25]:
#how about we get a summary of the base model?
base_model.summary()

Model: "efficientnetb0"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 rescaling_3 (Rescaling)        (None, None, None,   0           ['input_4[0][0]']                
                                3)                                                                
                                                                                                  
 normalization_3 (Normalization  (None, None, None,   7          ['rescaling_3[0][0]']            
 )                              3)                                                   

In [26]:
#how about a summary of our whole model?
model_0.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 efficientnetb0 (Functional)  (None, None, None, 1280)  4049571  
                                                                 
 global_average_pooling_laye  (None, 1280)             0         
 r (GlobalAveragePooling2D)                                      
                                                                 
 output_layer (Dense)        (None, 10)                12810     
                                                                 
Total params: 4,062,381
Trainable params: 12,810
Non-trainable params: 4,049,571
_________________________________________________________________
